In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys, os

def add_python_path(module_path):
    if os.path.abspath(module_path) not in sys.path:
        sys.path.append(os.path.abspath(module_path))
        print(f"python path: {os.path.abspath(module_path)} is added")
    else:
        print(f"python path: {os.path.abspath(module_path)} already exists")
    print("sys.path: ", sys.path)

module_path = ".."
add_python_path(module_path)
module_path = "../../.."
add_python_path(module_path)

python path: /home/ec2-user/SageMaker is added
sys.path:  ['/home/ec2-user/anaconda3/envs/python3/lib/python310.zip', '/home/ec2-user/anaconda3/envs/python3/lib/python3.10', '/home/ec2-user/anaconda3/envs/python3/lib/python3.10/lib-dynload', '', '/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages', '/home/ec2-user/SageMaker']
python path: /home is added
sys.path:  ['/home/ec2-user/anaconda3/envs/python3/lib/python310.zip', '/home/ec2-user/anaconda3/envs/python3/lib/python3.10', '/home/ec2-user/anaconda3/envs/python3/lib/python3.10/lib-dynload', '', '/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages', '/home/ec2-user/SageMaker', '/home']


## 1. Bedrock Client 생성

In [3]:
import json


#### Save reranker endpoint to Parameter Store

In [8]:
import boto3
from local_utils.ssm import parameter_store

# index_name = <your index>
index_name = "v1-faq-shinhan-bank"

region=boto3.Session().region_name
pm = parameter_store(region)
pm.put_params(
    key="opensearch_index_name",
    value=f'{index_name}',
    overwrite=True,
    enc=False
)

## Index 스키마 정의

In [14]:
rag_semantic = {
    "mappings": {
        "properties": {
            "tableName": {
                "type": "text",
                "fields": {
                          "english": {
                            "type": "text",
                            "analyzer": "english"},                
                            },
            },
            "question": {
                "type": "text",
                "fields": {
                          "english": {
                            "type": "text",
                            "analyzer": "english"},                
                            },                
            },
            "tableSchema": {
                "type": "text",
                "fields": {
                          "english": {
                            "type": "text",
                            "analyzer": "english"},                
                            },     #{"keyword": {"type": "keyword", "ignore_above": 256}},
            },
            "vector_field": {
                "type": "knn_vector",
                "dimension": 1024,
                "method": {"name": "hnsw", "space_type": "l2", "engine": "faiss"},
                "store": True,
            },

        }
    },
}

## 5. LangChain OpenSearch VectorStore 생성 
### 선수 조건


In [21]:
import boto3, json
def get_cfn_outputs(stackname, cfn):
    outputs = {}
    for output in cfn.describe_stacks(StackName=stackname)["Stacks"][0]["Outputs"]:
        outputs[output["OutputKey"]] = output["OutputValue"]
    return outputs

region_name = "us-west-2"

cfn = boto3.client("cloudformation", region_name)
kms = boto3.client("secretsmanager", region_name)

stackname = "opensearch-workshop"
cfn_outputs = get_cfn_outputs(stackname, cfn)

aos_credentials = json.loads(
    kms.get_secret_value(SecretId=cfn_outputs["OpenSearchSecret"])["SecretString"]
)

aos_host = cfn_outputs["OpenSearchDomainEndpoint"]

http_auth = (aos_credentials["username"], aos_credentials["password"])

# aos_client = OpenSearch(
#     hosts=[{"host": aos_host, "port": 443}],
#     http_auth=auth,
#     use_ssl=True,
#     verify_certs=True,
#     connection_class=RequestsHttpConnection,
# )

In [22]:
from local_utils.opensearch import opensearch_utils
os_client = opensearch_utils.create_aws_opensearch_client(
    region_name,
    aos_host,
    auth
)

index_name = "genai-demo-index-v1"
index_exists = opensearch_utils.check_if_index_exists(os_client, index_name)

if index_exists:
    opensearch_utils.delete_index(os_client, index_name)
else:
    print("Index does not exist")

### bedrock 연결

In [26]:
# We will be using the Titan Embeddings Model to generate our Embeddings.
from langchain.vectorstores import OpenSearchVectorSearch
from langchain.embeddings import BedrockEmbeddings
from langchain_community.chat_models import BedrockChat

import boto3
from botocore.config import Config

DB_NAME = "text2sql"
DB_FAISS_PATH = './vectorstore/db_faiss'

bedrock_region = athena_region = boto3.session.Session().region_name
retry_config = Config(retries = {'max_attempts': 100})
session = boto3.Session(region_name=bedrock_region)
bedrock = session.client('bedrock-runtime', region_name=bedrock_region, config=retry_config)

# model_name="Titan-Embeddings-G1"
llm_emb = BedrockEmbeddings(client=bedrock)


### data load

In [52]:
import pandas as pd
## openvectorsearch에 넣기 위한 documents 형태로 만들어야함
files = os.listdir('./data/rag')
res = []
for f in files:
    with open(f'./data/rag/{f}', 'rb') as ofp:
        df = json.load(ofp)
        res.extend(df)

In [65]:
df = pd.DataFrame(res)
df.to_csv('./data/concat_meta.csv', index=False)

In [67]:
import time
from langchain.indexes import VectorstoreIndexCreator
from langchain.vectorstores import FAISS
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter, SpacyTextSplitter

loader = CSVLoader(
    file_path="./data/concat_meta.csv",
    csv_args={
        "delimiter": ",",
    },
    source_column="tableName",
    encoding="utf-8"
)

documents_fsi = loader.load()

### index 생성

In [70]:
df

,tableName,question,tableSchema
0,Customer_Addresses,What is the total number of unique customers ...,customer_id|address_id|date_from|date_to
1,Customer_Addresses,Which customers have had multiple addresses a...,customer_id|address_id|date_from|date_to
2,Customer_Addresses,How many customers have had a change in their...,customer_id|address_id|date_from|date_to
3,Customer_Addresses,Can you identify the customers who have had t...,customer_id|address_id|date_from|date_to
4,Customer_Addresses,What is the distribution of address changes a...,customer_id|address_id|date_from|date_to
...,...,...,...
135,Department_Stores,Can customers reach out to a department store...,dept_store_id|dept_store_chain_id|store_name|s...
136,Department_Stores,Which department store locations are part of ...,dept_store_id|dept_store_chain_id|store_name|s...
137,Department_Stores,How many department store chains are represen...,dept_store_id|dept_store_chain_id|store_name|s...
138,Department_Stores,What is the geographic distribution of the de...,dept_store_id|dept_store_chain_id|store_name|s...


In [ ]:
%%time
# by default langchain would create a k-NN index and the embeddings would be ingested as a k-NN vector type
docsearch = OpenSearchVectorSearch.from_documents(
    index_name=index_name,
    documents=docs,
    embedding=llm_emb,
    opensearch_url=opensearch_domain_endpoint,
    http_auth=http_auth,
    bulk_size=10000,
    timeout=60
)

In [68]:
documents_fsi = loader.load()

In [74]:
is_bedrock_embeddings = True
if is_bedrock_embeddings:
    chunk_size = 2048
    chunk_overlap = 50
elif is_KoSimCSERobert:
    chunk_size = 800 # This is maxumum
    chunk_overlap = 0


text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size = chunk_size,
    chunk_overlap  = chunk_overlap,
    separators=["\n\n", "\n", ".", " ", ""],
    length_function = len,
)

docs = text_splitter.split_documents(documents_fsi)
print(f"Number of documents after split and chunking={len(docs)}")

Number of documents after split and chunking=140


### OpenSearch Client 생성

In [118]:
from local_utils.opensearch import opensearch_utils

In [119]:
aws_region = os.environ.get("AWS_DEFAULT_REGION", None)

os_client = opensearch_utils.create_aws_opensearch_client(
    aws_region,
    opensearch_domain_endpoint,
    http_auth
)